# Investigation of cross-resonance gate dynamical decoupling

This notebook aims at investigating the possibility to use dynamical decoupling on spectators qubits when a cross-resonance pulse is applied.

## Backend initialisation

In [1]:
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Load IBMQ Account data
from qiskit import IBMQ
IBMQ.load_account()

# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q-france', group='univ-montpellier', project='default') 
backend = provider.get_backend('ibmq_sydney')
properties = backend.properties()
configuration = backend.configuration()
defaults = backend.defaults()

## Dynamical decoupling sequences

We want to benchmark the most dynamical decoupling sequence possible in order to devise their effectiveness in protecting the quantum states.

In [5]:
from qiskit.ignis.mitigation.dd.sequence import BaseDynamicalDecouplingSequence

dd_sequences: List[BaseDynamicalDecouplingSequence] = list()
dd_names: List[str] = list()

### No dynamical-decoupling

A quick hack to be able to use "nothing" as a dynamical decoupling scheme (i.e. only delay).

In [6]:
from qiskit.ignis.mitigation.dd.components import DynamicalDecouplingDelayComponent

class NoDynamicalDecouplingSequence(BaseDynamicalDecouplingSequence):
    def __init__(self):    
        super(NoDynamicalDecouplingSequence, self).__init__(
            sequence=[DynamicalDecouplingDelayComponent()],
            relative_scales=[1]
        )

dd_sequences.append(NoDynamicalDecouplingSequence())
dd_names.append("none")

### Simple Hahn Echo Spin

The first sequence to be benchmarked is the Hahn-echo sequence, consisting of one $\pi$ pulse that rotates around the $X$-axis.

In [7]:
from qiskit.ignis.mitigation.dd.sequences import HahnSpinEchoDynamicalDecouplingSequence

hahn_sequence = HahnSpinEchoDynamicalDecouplingSequence(backend, add_pre_post_rotations=False)
dd_sequences.append(hahn_sequence)
dd_names.append("hahn")

### Carr-Purcell

The Carr-Purcell dynamical-decoupling sequence is a Hahn-echo sequence that is repeated several times. For this benchmark we empirically take $3$ repetitions.

In [8]:
from qiskit.ignis.mitigation.dd.sequences import CarrPurcellDynamicalDecouplingSequence

repetition_number = 3
cp_sequence = CarrPurcellDynamicalDecouplingSequence(
    backend, repetition_number=repetition_number, add_pre_post_rotations=False
)
dd_sequences.append(cp_sequence)
dd_names.append("cp")

### Carr-Purcell-Meiboom-Gill

Same as Carr-Purcel sequence but with $Y$ gates instead of $X$ gates.

In [9]:
from qiskit.ignis.mitigation.dd.sequences import (
    CPMGGateDynamicalDecouplingSequence, 
    CPMGPulseDynamicalDecouplingSequence
)

cpmg_gate_sequence = CPMGGateDynamicalDecouplingSequence(
    backend, repetition_number=repetition_number, add_pre_post_rotations=False
)
dd_sequences.append(cpmg_gate_sequence)
dd_names.append("cpmg_gate")

cpmg_pulse_sequence = CPMGPulseDynamicalDecouplingSequence(
    backend, repetition_number=repetition_number, add_pre_post_rotations=False
)
dd_sequences.append(cpmg_pulse_sequence)
dd_names.append("cpmg_pulse")

### $XY-4$ 

Applying $X$ and $Y$ gates in sequence twice to mitigate dephasing noise. Robust to more complex dephasing noise as the qubit state rotates around $2$ axis.

In [10]:
from qiskit.ignis.mitigation.dd.sequences import (
    XY4DynamicalDecouplingSequence, 
    XY4PulseDynamicalDecouplingSequence
)

xy4_gate_sequence = XY4DynamicalDecouplingSequence(backend, symmetric=True)
dd_sequences.append(xy4_gate_sequence)
dd_names.append("xy4_gate")

xy4_pulse_sequence = XY4PulseDynamicalDecouplingSequence(backend, symmetric=True)
dd_sequences.append(xy4_pulse_sequence)
dd_names.append("xy4_pulse")

### Knill Dynamical Decoupling (KDD)

The KDD scheme uses robust sequences of pulses to implement the $XY-4$ scheme with robust $X$ and $Y$ gates.

In [11]:
from qiskit.ignis.mitigation.dd.sequences import XY4KDDDynamicalDecouplingSequence

xy4kdd_sequence = XY4KDDDynamicalDecouplingSequence(backend, symmetric=True)
dd_sequences.append(xy4kdd_sequence)
dd_names.append("kdd")

### Uhrig Dynamical Decoupling (UDD)

Uhrig dynamical decoupling scheme is the only one using non-constant delays. It is theoretically very good as it cancels errors to higher orders.

In [12]:
from qiskit.ignis.mitigation.dd.sequences import (
    UhrigGateDynamicalDecouplingSequence, 
    UhrigPulseDynamicalDecouplingSequence
)

uhrig_gate_sequence = UhrigGateDynamicalDecouplingSequence(
    backend, repetition_number=repetition_number, add_pre_post_rotations=False
)
dd_sequences.append(xy4_gate_sequence)
dd_names.append("uhrig_gate")

uhrig_pulse_sequence = UhrigPulseDynamicalDecouplingSequence(
    backend, repetition_number=repetition_number, add_pre_post_rotations=False
)
dd_sequences.append(xy4_pulse_sequence)
dd_names.append("uhrig_pulse")

## Circuit creation

For each possible native CNOT we want to see if applying a given dynamical-decoupling scheme on possible spectator qubits lower down the impact of the CNOT on the spectator qubit.

As CNOTs are quite short, so some DD schemes might not be realisable in the given time.